<a href="https://colab.research.google.com/github/G5277/Gemini_api/blob/main/Chat_with_Documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 66.5 MB/s eta 0:00:00


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI



In [24]:
from IPython.display import display
from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•','*')
  return Markdown(textwrap.indent(text, '>', predicate = lambda _:True))

In [4]:
import google.generativeai as genai
from google.colab import userdata

In [5]:
import os
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key  = GOOGLE_API_KEY)

In [1]:
!sudo apt -y -qq install tesseract-ocr libtesseract-dev

!sudo apt-get -y -qq install poppler-utils libxml2-dev libxslt1-dev antiword unrtf tesseract-ocr pstotext flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

!pip install langchain langchain-community

The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 123594 files and directories

In [7]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

In [8]:
import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [9]:
warnings.filterwarnings("ignore")
# restart python kernal if issues with langchain import.

In [11]:
model = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", google_api_key = GOOGLE_API_KEY, temperature = 0.2, convert_system_message_to_human = True )

# PDF -- TEXT EXTRACTION

In [12]:
pdf_loader = PyPDFLoader("/content/NP_LAB_SLIDES.pdf")
pages = pdf_loader.load_and_split()
print(pages[3].page_content)
len(pages)

TCP/IP
* image is taken from “http://en.wikipedia.org/wiki/TCP/IP_model”*
Communication 
ChannelsNetwork Layer 
IPTransport Layer 
TCP or UDP
FTP, SMTP, …
CS556 - Distributed Systems Tutori al by Eleftherios Kosmas 4


100

# **RAG Pipeline: Embedding + Gemini (LLM)**

In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.llms import GooglePalm
from langchain.chains.question_answering import load_qa_chain


In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = "\n\n".join(str(p.page_content) for p in pages)
texts = text_splitter.split_text(context)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)
llm2 = GooglePalm(temperature=0, google_api_key = GOOGLE_API_KEY)
vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm2, # Pass the Google PaLM LLM instance
    chain_type="stuff", # Use a suitable chain type, 'stuff' is just an example
    retriever=vector_index,
    return_source_documents=True
)
question = "Explain the types of sockets?"
result = qa_chain({"query": question})
result["result"]



result["source_documents"]

[Document(page_content='Introduction to \nSockets Programming in C \nusing TCP/IP\nProfessor: Panagiota Fatourou\nTA: Eleftherios Kosmas\nCSD - May 2012\n\nIntroduction\nCS556 - Distributed Systems Tutori al by Eleftherios Kosmas 2\x84Computer Network\n\x89hosts, routers, \ncommunication channels\n\x84Hosts run applications\n\x84Routers forward information\n\x84Packets : sequence of bytes\n\x89contain control information\n\x89e.g. destination host\n\x84Protocol is an agreement\n\x89meaning of packets\n\x89structure and size of packets\ne.g. Hypertext Transfer Protocol \n(HTTP)\nHost\nRouter\nCommunication \nchannel\n\nProtocol Families - TCP/IP\nCS556 - Distributed Systems Tutori al by Eleftherios Kosmas 3\x84Several protocols for different problems\n)Protocol Suites or Protocol Families: TCP/IP\n\x84TCP/IP provides end-to-end connectivity specifying how data \nshould be \n\x89formatted, \n\x89addressed, \n\x89transmitted, \n\x89routed, and \n\x89received at the destination\n\x84can be

In [16]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)

In [17]:
llm2 = GooglePalm(temperature=0, google_api_key = GOOGLE_API_KEY)

In [18]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})
# Database : Chroma - using vector database as Retriever

In [19]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm2, # Pass the Google PaLM LLM instance
    chain_type="stuff", # Use a suitable chain type, 'stuff' is just an example
    retriever=vector_index,
    return_source_documents=True
)


In [39]:
question = "Explain the types of sockets ?"
result = qa_chain({"query": question})
result["result"]

'There are two types of sockets:\n1. Stream sockets (e.g. TCP)\n2. Datagram sockets (e.g. UDP)'

In [26]:
Markdown(result["result"])

There are two types of sockets:
1. Stream sockets (e.g. TCP)
2. Datagram sockets (e.g. UDP)

In [40]:
result["source_documents"]

[Document(page_content='Introduction to \nSockets Programming in C \nusing TCP/IP\nProfessor: Panagiota Fatourou\nTA: Eleftherios Kosmas\nCSD - May 2012\n\nIntroduction\nCS556 - Distributed Systems Tutori al by Eleftherios Kosmas 2\x84Computer Network\n\x89hosts, routers, \ncommunication channels\n\x84Hosts run applications\n\x84Routers forward information\n\x84Packets : sequence of bytes\n\x89contain control information\n\x89e.g. destination host\n\x84Protocol is an agreement\n\x89meaning of packets\n\x89structure and size of packets\ne.g. Hypertext Transfer Protocol \n(HTTP)\nHost\nRouter\nCommunication \nchannel\n\nProtocol Families - TCP/IP\nCS556 - Distributed Systems Tutori al by Eleftherios Kosmas 3\x84Several protocols for different problems\n)Protocol Suites or Protocol Families: TCP/IP\n\x84TCP/IP provides end-to-end connectivity specifying how data \nshould be \n\x89formatted, \n\x89addressed, \n\x89transmitted, \n\x89routed, and \n\x89received at the destination\n\x84can be

In [41]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end from the file. If the answer is not in file, just say that its not, don't try to make up an answer. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question : {question}
Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template) # Run Chain

qa_chain = RetrievalQA.from_chain_type(
    llm = llm2,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


In [48]:
question = "Summarise the pdf "
result = qa_chain({"query": question})
result["result"]

'This pdf introduces the basics of socket programming in Unix. It covers the following topics:\n- Creating a socket\n- Binding a socket to a port\n- Listening for connections\n- Accepting connections\n- Sending and receiving data\n- Closing a socket'

In [49]:
Markdown(result["result"])

This pdf introduces the basics of socket programming in Unix. It covers the following topics:
- Creating a socket
- Binding a socket to a port
- Listening for connections
- Accepting connections
- Sending and receiving data
- Closing a socket